# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Bedrock's Amazon Titan Text large model using the Boto3 API. 

The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text alongside their classification other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
To demonstrate the text generation capability of Amazon Bedrock, we will explore the use of Boto3 client to communicate with Amazon Bedrock API. We will demonstrate different configurations available as well as how simple input can lead to desired outputs.

#### Pattern
We will simply provide the Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock.jpg)

#### Use case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email.We will use the Amazon Titan Text Large model using the Amazon Bedrock API with Boto3 client. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock basics notebook](../00_Prerequisites/bedrock_basics.ipynb) notebook. ⚠️ ⚠️ ⚠️


In [2]:
import json
import os
import sys

import boto3
import botocore

boto3_bedrock = boto3.client('bedrock-runtime')

## Generate text

Following on the use case explained above, let's prepare an input for  the Amazon Bedrock service to generate an email. The full messages API docs:  https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html

In [13]:
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
max_tokens = 1000
top_p = 0.95
temperature = 0.1

# Prompt with user turn only.
user_message =  {"role": "user", "content":  """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""}
messages = [user_message]

In [14]:
body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "messages": messages, 
            "temperature": temperature,
            "top_p": top_p
        }
    )

response = boto3_bedrock.invoke_model(body=body, modelId=model_id)
response_body = json.loads(response.get('body').read())

In [15]:
response_body

{'id': 'msg_bdrk_011X93aYxBDgGpVAVeoR5vYY',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-haiku-20240307',
 'stop_sequence': None,
 'usage': {'input_tokens': 46, 'output_tokens': 303},
 'content': [{'type': 'text',
   'text': 'Here is a draft email from Bob, the Customer Service Manager, to the customer "John Doe" who provided negative feedback on the service provided by the customer support engineer:\n\nSubject: Addressing Your Feedback - Customer Support Experience\n\nDear John Doe,\n\nI am writing to you regarding the feedback you recently provided about your experience with our customer support team. As the Customer Service Manager, I want to personally apologize for the unsatisfactory service you received.\n\nYour feedback is extremely valuable to us, as it helps us identify areas where we can improve to better serve our customers. Please know that we take all customer concerns very seriously and are committed to providing a positive experience.\n\nI have reviewed 

In [16]:
print(response_body['content'][0]['text'])

Here is a draft email from Bob, the Customer Service Manager, to the customer "John Doe" who provided negative feedback on the service provided by the customer support engineer:

Subject: Addressing Your Feedback - Customer Support Experience

Dear John Doe,

I am writing to you regarding the feedback you recently provided about your experience with our customer support team. As the Customer Service Manager, I want to personally apologize for the unsatisfactory service you received.

Your feedback is extremely valuable to us, as it helps us identify areas where we can improve to better serve our customers. Please know that we take all customer concerns very seriously and are committed to providing a positive experience.

I have reviewed the details of your interaction, and I want to assure you that the issues you raised will be addressed. Our customer support team will be retrained on best practices for handling customer inquiries and concerns in a timely and professional manner.

Addi

##### Streaming Output Generation
Above is an example email generated by the Claude model by understanding the input request and using its inherent understanding of the different modalities. This request to the API is synchronous and waits for the entire output to be generated by the model.

Bedrock also supports that the output can be streamed as it is generated by the model in form of chunks. Below is an example of invoking the model with streaming option. `invoke_model_with_response_stream` returns a `ResponseStream` which you can read from.

_You may want to enable scrolling on your output cell below:_ 

In [18]:
response = boto3_bedrock.invoke_model_with_response_stream(
        body=body, modelId=model_id)

for event in response.get("body"):
    chunk = json.loads(event["chunk"]["bytes"])

    if chunk['type'] == 'message_delta':
        print(f"\nStop reason: {chunk['delta']['stop_reason']}")
        print(f"Stop sequence: {chunk['delta']['stop_sequence']}")
        print(f"Output tokens: {chunk['usage']['output_tokens']}")

    if chunk['type'] == 'content_block_delta':
        if chunk['delta']['type'] == 'text_delta':
            print(chunk['delta']['text'], end="")

Here is a draft email from Bob, the Customer Service Manager, to the customer "John Doe" who provided negative feedback on the service provided by the customer support engineer:

Subject: Addressing Your Feedback - Customer Service Improvement

Dear John Doe,

Thank you for taking the time to provide feedback on the service you received from our customer support team. I understand that your experience was not up to the high standards we strive for, and I want to personally apologize for the inconvenience this has caused you.

As the Customer Service Manager, I take all customer feedback very seriously, as it helps us identify areas for improvement within our organization. Please be assured that I have thoroughly reviewed the details of your interaction, and I have addressed the issues with the support engineer involved.

Moving forward, we will be providing additional training to our customer support team to ensure they are better equipped to handle complex inquiries and provide a more

The above helps to quickly get output of the model and let the service complete it as you read. This assists in use-cases where there are longer pieces of text that you request the model to generate. You can later combine all the chunks generated to form the complete output and use it for your use-case

## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You